# Tarea 4

**Envío del PR inicial:** viernes 28 de septiembre

**Aceptación del PR:** martes 9 de octubre

In [3]:
using Plots, LaTeXStrings

gr(grid=false)

Plots.GRBackend()

In [5]:
module Mapeos
using RecipesBase

export Mapeo, f

struct Mapeo
    F::Function
    x₀::Real
    n::Int
    xnn::Vector{Float64}
    x::Vector{Float64}
    y::Vector{Float64}
end

function itera_mapeo(F, x0; n_iters=100)
    xnn = Float64[x0]
    x = Float64[x0]; y = [-10000.]
    for i in 1:n_iters
        xn = xnn[end]
        Fn = F(xn)
        push!(xnn, Fn)
        push!(x, xn)
        push!(y, Fn)
        push!(x, Fn)
        push!(y, Fn)
    end
    xnn, (x, y)
end

function Mapeo(F::Function, x₀::Real, n::Int)
    xnn, x_y = itera_mapeo(F, x₀, n_iters=n)
    Mapeo(F, x₀, n, xnn, x_y...)
end

@userplot grafica_mapeo

@recipe function f(p::grafica_mapeo)
    if length(p.args) != 2 || !(typeof(p.args[1]) <: Mapeo) # Revisar esto, la primer condición no se evalúa
        error("grafica_mapeo recibe dos parámetros, el mapeo y el número de pasos.")
    end
    mapeo = p.args[1]
    n = p.args[2]
    F, x, y = mapeo.F, mapeo.x, mapeo.y
    rango_x = [minimum(x), maximum(x) + 2]
    rango_y = [minimum(y[2:end]) - 2, maximum(y) + 2]
    xx = linspace(rango_x[1], rango_x[2], 1000)
    xlims := rango_x
    ylims := rango_y
    @series begin
        seriestype := :line
        label := "F(x)"
        xx, x -> F(x)
    end
    @series begin
        seriestype := :line
        label := "y=x"
        xx, x -> x
    end
    @series begin
        seriestype := :path
        ls := :dash
        marker := (3, .5, :dot)
        label := "Trayectoria"
        x[1:n], y[1:n]
    end
end

end

Mapeos

## 1

Para el mapeo $Q_c(x) = x^2 + c$, ¿cuál es el dominio $\cal{D}\subset \mathbb{R}$ de $Q_c(x)$ tal que la órbita generada por una condición inicial $x_0\in \cal{D}$ *no* escapa a infinito a ningún tiempo?

In [6]:
Q_c(x, c) = x^2 + c

Q_c (generic function with 1 method)

In [ ]:
pplus(c) = 0.5*(1+sqrt(1-4*c))
pminus(c) = 0.5*(1-sqrt(1-4*c))

domc = -2:1/64:1/4

plot(-2:1/64:1/4, pplus, xaxis=("c", (-2,1)), yaxis=("p_\pm(c)"), 
    label="p_+(c)", color=:red)
plot!(-3/4:1/64:1/4, pminus, label="p_-(c)", color=:blue)
plot!(-2:1/64:-3/4, pminus, label="", color=:red, line=(:dash))
title!("Fig. 2")

## 2

- Obtengan el diagrama de bifurcación que se muestra en la Fig. 2 del mapeo $Q_c(x)$ a partir de la iteración de una condición inicial.

- Caractericen la cuenca de atracción de $p_-(c)$ cuando el punto es estable.


## 3

Para el mapeo $F_\lambda(x) = \lambda x(1-x)$, ¿qué podemos decir de la bifurcación que ocurre en $\lambda_0=1$? ¿Es o no una bifurcación de silla-nodo?

## 4

- Obtengan numéricamente el diagrama de bifurcación del mapeo $Q_c(x)$ a partir de la iteración de una condición inicial en el intervalo de parámetros $-2<c<1/4$. Para esto se necesita iterar un gran número de veces (hasta llegar al ciclo periódico) condiciones iniciales, y repetir esto para *muchos* valores de $c$ en el intervalo. Vale la pena hacer notar que, si hay ciclos periódicos, *a priori* no conocen su periodicidad.

- Para ciertos valores de $c$ característicos, muestren el análisis gráfico del mapeo $Q_c(x)$ a fin de entender los resultados.


## 5

El objetivo de este ejercicio es relacionar, la velocidad de convergencia con que un punto fijo (o una órbita periódica, en el caso de los dos últimos incisos) atraen a puntos suficientemente cercanos, con la derivada del mapeo en el punto fijo (o ciclo periódico). La idea es, entonces, calcular primero el punto fijo y, después, medir cómo la distancia de los iterados sucesivos (de una condición inicial $x_0$) al punto fijo se comporta en el tiempo, para los siguientes mapeos:

- $F(x) = x^2+0.25$

- $F(x) = 3x(1-x)$

- $F(x) = \exp(x-1)$

- $F(x) = x^2 - 1.25$

- $F(x) = \exp(x+1)$

Deberán resolver algunas cosas intermedias. Por ejemplo, ¿qué tanto deben acercarse al punto fijo, a fin de evitar ruido numérico? ¿Qué hay que hacer en el caso en que el punto tenga una estabilidad neutra, i.e., no sea no atractivo ni repulsivo?

En los dos últimos incisos, el interés está en los ciclos de periodo 2.